# Adivinhando os dígitos

## Preparando os inputs

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import time

In [2]:
def cos_sin(W, i, j, k):
    # ajuste de contagem
    i = i - 1
    j = j - 1
    k = k - 1
    
    if abs(W[i,k]) > abs(W[j,k]):
        t = -W[j,k]/W[i,k]
        c = 1/math.sqrt(1 + t**2)
        s = c*t
    else:
        t = -W[i, k]/W[j, k]
        s = 1/math.sqrt(1 + t**2)
        c = s*t    
    return c, s

def rot_Givens(W, n, m, i, j, c, s):
    # ajuste de contagem
    i = i - 1
    j = j - 1
    
    W[i,0:m], W[j,0:m] = c*W[i,0:m] - s*W[j,0:m], s*W[i,0:m] + c*W[j,0:m]

In [3]:
def solve_multi(W, A, n, m, p):
    eps = 10e-10
    for k in range(1, p + 1):
        for j in range(n, k, -1):
            i = j - 1
            if abs(W[j - 1, k - 1]) > eps:
                cos, sin = cos_sin(W, i, j, k)
                rot_Givens(W, n, p, i, j, cos, sin)
                rot_Givens(A, n, m, i, j, cos, sin)
    
    for k in range(p, 0, -1):
        for j in range(1, m + 1):
            soma = 0
            for i in range(k+1,p+1):
                soma += W[k-1,i-1]*A[i-1,j-1]
            A[k-1,j-1] = (A[k-1,j-1] - soma)/W[k-1,k-1]

# Teste inicial

In [8]:
# Abrindo o caso mais simples 100 e 5
def read_all_digits(ndig_treino, p):
    all_dig = []
    
    for digito in range(10):
        filename = 'dig_treino/W_{}_{}_{}.txt'.format(digito, ndig_treino, p)
    
        with open(filename) as f:
            time = f.readline()
            Wd = np.loadtxt(f)
            all_dig.append(Wd)
    
    return all_dig
        
class PPTO: # Palpitômetro
    
    def __init__(self):
        
        # abrindo os arquivos
        with open('dados_mnist/test_images.txt') as f:
            self.A = np.loadtxt(f)
        with open('dados_mnist/test_index.txt') as f:
            self.asw = np.loadtxt(f, dtype=np.int8)
        
        # setando variáveis
        self.n, self.m = self.A.shape
        self.p = 0
        
        # vetores auxiliares
        self.D = np.zeros(self.m, dtype=np.int8)
        self.E = np.zeros(self.m)
        
        # todos os erros
        self.all_E = np.zeros((10, self.m))
        self.onde_errou = []

        # setando o contador
        self.count_dig = {}
        for digito in self.asw:
            sdigito = str(digito)
            
            if sdigito not in self.count_dig:
                self.count_dig[sdigito] = 1
            
            else:
                self.count_dig[sdigito] += 1

    def MMQS(self, Wds):
        self.n, self.p = Wds[0].shape # todos os 9 Wd's têm a mesma forma (shape)
        
        num_dig = len(Wds)
        for d in range(num_dig):
            # setando as variáveis
            Wd = Wds[d].copy()
            cpA = self.A.copy()
            
            # resolvendo o sistema simultâneo
            solve_multi(Wd, cpA, self.n, self.m, self.p)
            
            WH = np.dot(Wds[d], cpA[:self.p,:])
            C = np.subtract(self.A, WH)
            
            # calculando o erro de cada coluna
            for col in range(self.m):
                err = 0.0
                
                for row in range(self.n):
                    err += (C[row, col])**2
                    
                err = math.sqrt(err)
                
                # coletando todos os erros
                self.all_E[d, col] = err
                
                # gerando os palpites
                if d == 0:
                    self.E[col] = err
                
                elif err < self.E[col]:
                    self.E[col] = err
                    self.D[col] = d
    
    def generate_report(self, ndig_treino):
        
        # inicializando o contador de acertos
        correct = 0
        correct_dig = {}
        for num in range(10):
            snum = str(num)
            correct_dig[snum] = 0
        
        # contando os acertos
        for row in range(self.m):
            if self.D[row] == self.asw[row]:
                correct += 1
                correct_dig[str(self.D[row])] += 1
            else:
                self.onde_errou.append(int(row))
            
        # gerando o relatório
        with open('reports/report_{}_{}.txt'.format(ndig_treino, self.p), 'w+') as f:
            f.write('Relatório do caso p = {} e ndig_treino = {}\n\n'.format(self.p, ndig_treino))
            f.write('Percentual total de acertos: {:.2f}%\n\n'.format((correct/self.m)*100))
            
            for dig, right in correct_dig.items():
                f.write('Digito ' + dig + '\n')
                total = self.count_dig[dig]
                f.write('Acertos: {}/{}\n'.format(right, total))
                f.write('Percentual de acertos: {:.2f}%\n\n'.format((right/total)*100))
        
        with open('erros/erros_{}_{}.txt'.format(ndig_treino, self.p), 'w+') as f:
            np.savetxt(f, self.all_E)
        with open('erros/onde_errou_{}_{}.txt'.format(ndig_treino, self.p), 'w+') as f:
            np.savetxt(f, self.onde_errou)

In [9]:
pp = PPTO()

# Caso teste

In [10]:
from datetime import datetime

# ndig_treinos = [100, 1000, 4000]
# ps = [5, 10, 15]

ndig_treinos = [100, 1000, 4000]
ps = [5, 10, 15]

print('Horário de começo: {}\n'.format(datetime.now().time()))

for ndig_treino in ndig_treinos:
    for p in ps:
        print('Caso ndig_treino {} com p {}\n'.format(ndig_treino, p))
        start = time.time()

        Wds = read_all_digits(ndig_treino, p)
        pp.MMQS(Wds)

        elapsed_time = (time.time() - start)
        print('Tempo gasto para o MMQ: {}\n'.format(elapsed_time))

        pp.generate_report(ndig_treino)

print('Acabou às: {}'.format(datetime.now().time()))

Horário de começo: 23:43:08.388596

Caso ndig_treino 100 com p 5

Tempo gasto para o MMQ: 82.34726786613464

Caso ndig_treino 100 com p 10



<ipython-input-2-0fe0c9a09a57>:12: RuntimeWarning: invalid value encountered in double_scalars
  t = -W[i, k]/W[j, k]
<ipython-input-2-0fe0c9a09a57>:22: RuntimeWarning: invalid value encountered in multiply
  W[i,0:m], W[j,0:m] = c*W[i,0:m] - s*W[j,0:m], s*W[i,0:m] + c*W[j,0:m]
<ipython-input-3-5d83d9029389>:16: RuntimeWarning: invalid value encountered in double_scalars
  A[k-1,j-1] = (A[k-1,j-1] - soma)/W[k-1,k-1]


Tempo gasto para o MMQ: 91.46226453781128

Caso ndig_treino 100 com p 15

Tempo gasto para o MMQ: 102.61899995803833

Caso ndig_treino 1000 com p 5

Tempo gasto para o MMQ: 86.58755040168762

Caso ndig_treino 1000 com p 10

Tempo gasto para o MMQ: 80.81926465034485

Caso ndig_treino 1000 com p 15

Tempo gasto para o MMQ: 91.01249527931213

Caso ndig_treino 4000 com p 5

Tempo gasto para o MMQ: 73.42191648483276

Caso ndig_treino 4000 com p 10

Tempo gasto para o MMQ: 79.30150866508484

Caso ndig_treino 4000 com p 15

Tempo gasto para o MMQ: 90.18512797355652

Acabou às: 23:56:10.134507
